## practice pi-heaan
- Lets make logistic algorithm
- This algorithm is not perfect.
- And this algorithm cannot be applied to all data for logistic regression.
- Please consider this as an exercise to practice how to make an algorithm using pi-heaan.

In [11]:
import piheaan as heaan
from piheaan.math import approx
import time

import pandas as pd
import numpy as np
import math

In [12]:
start_time = time.time()
params = heaan.ParameterPreset.FGb
context = heaan.make_context(params)
heaan.make_bootstrappable(context)

# Load pre-exisisting key
key_file_path = "./keys"

sk = heaan.SecretKey(context,key_file_path+"/secretkey.bin") # load sk
pk = heaan.KeyPack(context, key_file_path+"/") # load pk
pk.load_enc_key()
pk.load_mult_key()

eval = heaan.HomEvaluator(context,pk)
dec = heaan.Decryptor(context)
enc = heaan.Encryptor(context)

log_slots = 15
num_slots = 2**log_slots

In [13]:
def normalize_data(arr):
    S = 0
    for i in range(len(arr)):
        S += arr[i]
    return [arr[i] / S for i in range(len(arr))]

In [14]:
def step(learning_rate, ctxt_X, ctxt_Y, ctxt_beta, n, log_slots, context, eval):
    '''
    ctxt_X, ctxt_Y : data for training
    ctxt_beta : initial value beta
    n : the number of row in train_data
    '''
    ctxt_rot = heaan.Ciphertext(context)
    ctxt_tmp = heaan.Ciphertext(context)
    
    ## step1
    # beta0
    ctxt_beta0 = heaan.Ciphertext(context)
    eval.left_rotate(ctxt_beta, 8*n, ctxt_beta0)
    
    # compute  ctxt_tmp = beta1*x1 + beta2*x2 + ... + beta8*x8 + beta0
    ctxt_tmp = heaan.Ciphertext(context)
    eval.mult(ctxt_beta, ctxt_X, ctxt_tmp)
    
    for i in range(3):
        eval.left_rotate(ctxt_tmp, n*2**(2-i), ctxt_rot)
        eval.add(ctxt_tmp, ctxt_rot, ctxt_tmp)
    eval.add(ctxt_tmp, ctxt_beta0, ctxt_tmp)
    
    msg_mask = heaan.Message(log_slots)
    for i in range(n):
        msg_mask[i] = 1
    eval.mult(ctxt_tmp, msg_mask, ctxt_tmp)
    
    ## step2
    # compute sigmoid
    approx.sigmoid(eval, ctxt_tmp, ctxt_tmp, 8.0)
    eval.bootstrap(ctxt_tmp, ctxt_tmp)
    msg_mask = heaan.Message(log_slots)
    # if sigmoid(0) -> return 0.5
    for i in range(n, num_slots):
        msg_mask[i] = 0.5
    eval.sub(ctxt_tmp, msg_mask, ctxt_tmp)
    
    ## step3
    # compute  (learning_rate/n) * (y_(j) - p_(j))
    ctxt_d = heaan.Ciphertext(context)
    eval.sub(ctxt_Y, ctxt_tmp, ctxt_d)
    eval.mult(ctxt_d, learning_rate / n, ctxt_d)
    
    eval.right_rotate(ctxt_d, 8*n, ctxt_tmp) # for beta0
    for i in range(3):
        eval.right_rotate(ctxt_d, n * 2**i, ctxt_rot)
        eval.add(ctxt_d, ctxt_rot, ctxt_d)
    eval.add(ctxt_d, ctxt_tmp, ctxt_d)
    
    ## step4
    # compute  (learning_rate/n) * (y_(j) - p_(j)) * x_(j)
    ctxt_X_j = heaan.Ciphertext(context)
    msg_X0 = heaan.Message(log_slots)
    for i in range(8*n, 9*n):
        msg_X0[i] = 1
    eval.add(ctxt_X, msg_X0, ctxt_X_j)
    eval.mult(ctxt_X_j, ctxt_d, ctxt_d)
    
    ## step5
    # compute  Sum_(all j) (learning_rate/n) * (y_(j) - p_(j)) * x_(j)
    for i in range(9):
        eval.left_rotate(ctxt_d, 2**(8-i), ctxt_rot)
        eval.add(ctxt_d, ctxt_rot, ctxt_d)
    msg_mask = heaan.Message(log_slots)
    for i in range(9):
        msg_mask[i * n] = 1
    eval.mult(ctxt_d, msg_mask, ctxt_d)

    for i in range(9):
        eval.right_rotate(ctxt_d, 2**i, ctxt_rot)
        eval.add(ctxt_d, ctxt_rot, ctxt_d)

    ## step6
    # update beta
    eval.add(ctxt_beta, ctxt_d, ctxt_d)
    return ctxt_d

- Next function is for evaluation

In [15]:
def compute_sigmoid(ctxt_X, ctxt_beta, n, log_slots, eval, context, num_slots):
    '''
    ctxt_X : data for evaluation
    ctxt_beta : estimated beta from function 'step'
    n : the number of row in test_data
    '''
    ctxt_rot = heaan.Ciphertext(context)
    ctxt_tmp = heaan.Ciphertext(context)
    
    # beta0
    ctxt_beta0 = heaan.Ciphertext(context)
    eval.left_rotate(ctxt_beta, 8*n, ctxt_beta0)
    
    # compute x * beta + beta0
    ctxt_tmp = heaan.Ciphertext(context)
    eval.mult(ctxt_beta, ctxt_X, ctxt_tmp)
    
    for i in range(3):
        eval.left_rotate(ctxt_tmp, n*2**(2-i), ctxt_rot)
        eval.add(ctxt_tmp, ctxt_rot, ctxt_tmp)
    eval.add(ctxt_tmp, ctxt_beta0, ctxt_tmp)
    
    msg_mask = heaan.Message(log_slots)
    for i in range(n):
        msg_mask[i] = 1
    eval.mult(ctxt_tmp, msg_mask, ctxt_tmp)
    
    # compute sigmoid
    approx.sigmoid(eval, ctxt_tmp, ctxt_tmp, 8.0)
    eval.bootstrap(ctxt_tmp, ctxt_tmp)
    msg_mask = heaan.Message(log_slots)
    for i in range(n, num_slots):
        msg_mask[i] = 0.5
    eval.sub(ctxt_tmp, msg_mask, ctxt_tmp)
    
    return ctxt_tmp

#### train

In [16]:
# 현재 데이터 처리를 통해 수정할 수 있는 부분 - train datatset 관련해서 변수 수정하기
# 내부 구조를 어떻게 바꿔서 출력을 다르게 할 것인가?를 생각할 수 있어야 한다.

csv_train = pd.read_csv('train_0528.csv')
df = pd.DataFrame(csv_train)

# preprocessing data
# convert only on columns that are not 0 ~ 1 to values 0 ~ 1
train_n = df.shape[0]
X = [0] * 18
X[0] = list(df['Gender'].values)
X[1] = list(df['Own_car'].values)
X[2] = list(df['Own_property'].values)
X[3] = list(df['Work_phone'].values)
X[4] = list(df['Phone'].values)
X[5] = list(df['Email'].values)
X[6] = list(df['Unemployed'].values)
X[7] = normalize_data(df['Num_children'].values)
X[8] = normalize_data(df['Num_family'].values)
X[9] = normalize_data(df['Account_length'].values)
X[10] = normalize_data(df['Total_income'].values)
X[11] = normalize_data(df['Age'].values)
X[12] = normalize_data(df['Years_employed'].values)
X[13] = normalize_data(df['Income_type'].values)
X[14] = normalize_data(df['Education_type'].values)
X[15] = normalize_data(df['Family_status'].values)
X[16] = normalize_data(df['Housing_type'].values)
X[17] = normalize_data(df['Occupation_type'].values)
Y = list(df['Target'].values)

msg_X = heaan.Message(log_slots)
ctxt_X = heaan.Ciphertext(context)
for i in range(18):
    for j in range(train_n):
        msg_X[train_n*i + j] = X[i][j]
enc.encrypt(msg_X, pk, ctxt_X)

msg_Y = heaan.Message(log_slots)
ctxt_Y = heaan.Ciphertext(context)
for j in range(train_n):
    msg_Y[j] = Y[j]
enc.encrypt(msg_Y, pk, ctxt_Y)

In [17]:
# initial value beta
beta = 2 * np.random.rand(19) - 1 # 동일함
print("beta : ", beta)
print()
msg_beta = heaan.Message(log_slots)
ctxt_beta = heaan.Ciphertext(context)

for i in range(18):
    for j in range(train_n):
        msg_beta[train_n*i + j] = beta[i+1]
for j in range(train_n):
    msg_beta[8*train_n + j] = beta[0]
    
enc.encrypt(msg_beta, pk, ctxt_beta)
print("msg_beta : ", msg_beta)

beta :  [-0.91331118  0.17221093  0.80508234  0.26989187  0.76006654  0.23148785
 -0.79843854 -0.05768929  0.51319418  0.61047499 -0.04085157  0.55573854
  0.39453243 -0.54759689  0.51679901 -0.41206088 -0.79304773  0.07290013
  0.86954716]

msg_beta :  [ (0.172211+0.000000j), (0.172211+0.000000j), (0.172211+0.000000j), (0.172211+0.000000j), (0.172211+0.000000j), ..., (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]


In [18]:
# randomly assign learning_rate
learning_rate = 0.2 # 임의의 값인가?
num_steps = 100

ctxt_next = heaan.Ciphertext(context)
eval.add(ctxt_beta, 0, ctxt_next)
for i in range(num_steps):
    print("=== Step", i, "===")
    # estimate beta_hat using function 'step' for 100 iteration
    ctxt_next = step(0.2, ctxt_X, ctxt_Y, ctxt_next, train_n, log_slots, context, eval)
    # print(ctxt_next)

=== Step 0 ===
=== Step 1 ===
=== Step 2 ===
=== Step 3 ===
=== Step 4 ===
=== Step 5 ===
=== Step 6 ===
=== Step 7 ===
=== Step 8 ===
=== Step 9 ===
=== Step 10 ===
=== Step 11 ===
=== Step 12 ===
=== Step 13 ===
=== Step 14 ===
=== Step 15 ===
=== Step 16 ===
=== Step 17 ===
=== Step 18 ===
=== Step 19 ===
=== Step 20 ===
=== Step 21 ===
=== Step 22 ===
=== Step 23 ===
=== Step 24 ===
=== Step 25 ===
=== Step 26 ===
=== Step 27 ===
=== Step 28 ===
=== Step 29 ===
=== Step 30 ===
=== Step 31 ===
=== Step 32 ===
=== Step 33 ===
=== Step 34 ===
=== Step 35 ===
=== Step 36 ===
=== Step 37 ===
=== Step 38 ===
=== Step 39 ===
=== Step 40 ===
=== Step 41 ===
=== Step 42 ===
=== Step 43 ===
=== Step 44 ===
=== Step 45 ===
=== Step 46 ===
=== Step 47 ===
=== Step 48 ===
=== Step 49 ===
=== Step 50 ===
=== Step 51 ===
=== Step 52 ===
=== Step 53 ===
=== Step 54 ===
=== Step 55 ===
=== Step 56 ===
=== Step 57 ===
=== Step 58 ===
=== Step 59 ===
=== Step 60 ===
=== Step 61 ===
=== Step 62 ===
==

#### evaluation

In [19]:
# prepare test data for evaluation
# convert only on columns that are not 0 ~ 1 to values 0 ~ 1
csv_test = pd.read_csv('test_0528.csv')
df = pd.DataFrame(csv_test)
test_n = df.shape[0]

X = [0] * 18
X[0] = list(df['Gender'].values)
X[1] = list(df['Own_car'].values)
X[2] = list(df['Own_property'].values)
X[3] = list(df['Work_phone'].values)
X[4] = list(df['Phone'].values)
X[5] = list(df['Email'].values)
X[6] = list(df['Unemployed'].values)
X[7] = normalize_data(df['Num_children'].values)
X[8] = normalize_data(df['Num_family'].values)
X[9] = normalize_data(df['Account_length'].values)
X[10] = normalize_data(df['Total_income'].values)
X[11] = normalize_data(df['Age'].values)
X[12] = normalize_data(df['Years_employed'].values)
X[13] = normalize_data(df['Income_type'].values)
X[14] = normalize_data(df['Education_type'].values)
X[15] = normalize_data(df['Family_status'].values)
X[16] = normalize_data(df['Housing_type'].values)
X[17] = normalize_data(df['Occupation_type'].values)
Y = list(df['Target'].values)

msg_X_test = heaan.Message(log_slots)
ctxt_X_test = heaan.Ciphertext(context)
for i in range(18):
    for j in range(test_n):
        msg_X_test[test_n*i + j] = X[i][j]
enc.encrypt(msg_X_test, pk, ctxt_X_test)

In [20]:
# accuracy
ctxt_infer = compute_sigmoid(ctxt_X_test, ctxt_next, test_n, log_slots, eval, context, num_slots)

res = heaan.Message(log_slots)
dec.decrypt(ctxt_infer, sk, res)
cnt = 0
for i in range(test_n):
    if res[i].real >= 0.6:
        if Y[i] == 1:
            cnt += 1
    else:
        if Y[i] == 0:
            cnt += 1
print("Accuracy : ", cnt / test_n)
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Accuracy :  0.86
Execution time: 6.18 seconds


In [ ]:
from flask import Flask, render_template, request
from IPython.display import clear_output

app = Flask(__name__)

# 모델 관련 코드 (생략)

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        # 사용자 입력 받기
        inputs = []
        for i in range(18):
            input_value = request.form[f'input{i}']
            inputs.append(float(input_value))
        
        # 입력값 암호화
        msg_input = heaan.Message(log_slots)
        ctxt_input = heaan.Ciphertext(context)
        for i in range(18):
            msg_input[i] = inputs[i]
        enc.encrypt(msg_input, pk, ctxt_input)
        
        # 모델에 입력하여 결과 계산
        ctxt_result = compute_sigmoid(ctxt_input, ctxt_next, 1, log_slots, eval, context, num_slots)
        
        res = heaan.Message(log_slots)
        
        # 입력값 복호화
        dec.decrypt(ctxt_result, sk, res)
        
        print("Result:", res[0].real)  # 디버깅 코드 추가
        
        result = res[0].real
        
        # 결과에 따라 메시지 출력
        if result >= 0.5:
            result_message = "신용카드 발급 가능합니다."
        else:
            result_message = "신용카드 발급이 불가합니다."
        
        # 사용자 입력값 출력
        user_input_str = ", ".join(str(x) for x in inputs)
        
        return render_template('index.html', result=result_message, user_input=user_input_str)
    
    return render_template('index.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)
    clear_output(wait=True)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2024 00:35:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2024 00:35:20] "POST / HTTP/1.1" 200 -


Result: 0.0016607935025756524
Result: 0.0
Result: 0.0
Result: 0.0
Result: 0.0
Result: 0.0
Result: 0.0


127.0.0.1 - - [29/May/2024 00:39:16] "GET / HTTP/1.1" 200 -
